In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:
# Load SenticNet Dataset
def load_senticnet(file_path):
    senticnet_df = pd.read_csv(file_path, sep=",")
    concept_dict = {}

    for _, row in senticnet_df.iterrows():
        concept = row['CONCEPT']
        primary_emotion = row['PRIMARY EMOTION']
        polarity_value = row['POLARITY VALUE']
        semantics = row['SEMANTICS'].split() if pd.notna(row['SEMANTICS']) else []
        concept_dict[concept] = {
            'primary_emotion': primary_emotion,
            'polarity_value': polarity_value,
            'semantics': semantics[:3]  # Top 3 related concepts
        }
    return concept_dict

# Inject Knowledge into Sentence
def inject_knowledge(sentence, senticnet):
    """
    Inject knowledge from SenticNet into a sentence and create a structured sentence tree.
    """
    words = sentence.split()
    sentence_tree = []

    for word in words:
        # Fetch knowledge for the word
        knowledge = senticnet.get(word, {})
        if knowledge:
            # Structure: word + related semantics
            node = {
                'word': word,
                'semantics': [
                    {'word': semantic_word, 'knowledge': senticnet.get(semantic_word, {})}
                    for semantic_word in knowledge.get('semantics', [])
                ]
            }
        else:
            # No knowledge found
            node = {'word': word, 'semantics': []}

        sentence_tree.append(node)
    return sentence_tree


In [4]:
#for removing noise

def assign_soft_positions_and_reorder(sentence_tree):
    tokens = []
    soft_positions = []
    position = 0

    # Step 1: Add main words
    main_tokens = []
    main_positions = []

    # Step 2: Collect semantics separately
    semantics_tokens = []
    semantics_positions = []

    for node in sentence_tree:
        # Add main word
        main_tokens.append(node['word'])
        main_positions.append(position)

        # Collect semantics (related words)
        for semantic in node['semantics']:
            semantics_tokens.append(semantic['word'])
            semantics_positions.append(position)

        position += 1

    # Step 3: Reorder by appending semantics after main tokens
    tokens = main_tokens + semantics_tokens
    soft_positions = main_positions + semantics_positions

    return tokens, soft_positions



#Visible Matrix
def create_visible_matrix_from_tree(sentence_tree):
    total_tokens = sum(1 + len(node['semantics']) for node in sentence_tree)
    visible_matrix = torch.zeros((total_tokens, total_tokens))

    idx = 0
    for i, node in enumerate(sentence_tree):
        visible_matrix[idx, idx] = 1  # Main word visible to itself
        child_start_idx = idx + 1

        # Main word visible to its semantics
        for j, semantic in enumerate(node['semantics']):
            visible_matrix[idx, child_start_idx + j] = 1
            visible_matrix[child_start_idx + j, idx] = 1

        idx += 1 + len(node['semantics'])

    return visible_matrix


def pad_visible_matrix(visible_matrix, max_length):
    """Pad visible matrix to max_length."""
    padded_matrix = torch.zeros((max_length, max_length))
    current_length = visible_matrix.size(0)
    padded_matrix[:current_length, :current_length] = visible_matrix
    return padded_matrix


In [5]:
class K_BERTDataset(Dataset):
    def __init__(self, data, tokenizer, senticnet, max_length=128):
        self.data = data  # Accept DataFrame directly
        self.tokenizer = tokenizer
        self.senticnet = senticnet
        self.max_length = max_length

        # Create a label mapping
        unique_labels = sorted(self.data['label'].unique())
        self.label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
        self.data['label'] = self.data['label'].map(self.label_mapping)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['content']
        label = self.data.iloc[idx]['label']

        # Inject SenticNet knowledge
        sentence_tree = inject_knowledge(sentence, self.senticnet)
        
        tokens, soft_positions = assign_soft_positions_and_reorder(sentence_tree)
        soft_positions = torch.tensor(soft_positions[:self.max_length], dtype=torch.long)
            
        # Tokenize inputs
        inputs = self.tokenizer(
            tokens,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            is_split_into_words=True
        )

        # Create visible matrix
        visible_matrix = create_visible_matrix_from_tree(sentence_tree)

        # Pad visible matrix to match max_length
        padded_visible_matrix = pad_visible_matrix(visible_matrix, self.max_length)

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'visible_matrix': padded_visible_matrix,
            'soft_positions': soft_positions,
            'label': torch.tensor(label, dtype=torch.long)
        }


In [6]:
def expand_visible_matrix(visible_matrix, num_heads):
    batch_size, seq_len, seq_len_2 = visible_matrix.size()
    expanded_matrix = visible_matrix.unsqueeze(1).repeat(1, num_heads, 1, 1)
    return expanded_matrix.view(batch_size * num_heads, seq_len, seq_len)

class MaskedSelfAttention(nn.Module):
    def __init__(self, hidden_size, num_attention_heads):
        super().__init__()
        self.num_attention_heads = num_attention_heads
        self.attention = nn.MultiheadAttention(hidden_size, num_attention_heads)
    
    def forward(self, inputs, attention_mask, visible_matrix):
        batch_size, seq_len, hidden_size = inputs.size()

        # Incorporate attention_mask into visible_matrix:
        # attention_mask: [batch_size, seq_len], 1 for real tokens, 0 for padded
        # Make broadcastable
        valid_tokens = attention_mask.unsqueeze(-1) * attention_mask.unsqueeze(1)
        combined_matrix = visible_matrix * valid_tokens

        # Expand for multi-head
        combined_matrix = expand_visible_matrix(combined_matrix, self.num_attention_heads)

        # Create attn_mask with a large negative number instead of -inf
        attn_mask = torch.zeros_like(combined_matrix, dtype=torch.float)
        attn_mask[combined_matrix == 0] = -1e9  # large negative number

        # inputs: [batch_size, seq_len, hidden_size] -> [seq_len, batch_size, hidden_size]
        inputs = inputs.permute(1, 0, 2)

        # Apply multi-head attention
        # IMPORTANT: multihead attention expects attn_mask to be [L, S] or [N*num_heads, L, S].
        # We have [N*num_heads, seq_len, seq_len], which should be correct since we repeated for heads.
        outputs, _ = self.attention(inputs, inputs, inputs, attn_mask=attn_mask)
        
        # [seq_len, batch_size, hidden_size] -> [batch_size, seq_len, hidden_size]
        outputs = outputs.permute(1, 0, 2)
        return outputs


In [7]:
class K_RoBERTa(nn.Module):
    def __init__(self, transformer, hidden_size, num_attention_heads, num_labels, max_length=128):
        super().__init__()
        self.transformer = transformer  # Use RobertaForSequenceClassification
        self.masked_attention = MaskedSelfAttention(hidden_size, num_attention_heads)
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.soft_position_embeddings = nn.Embedding(max_length, hidden_size)

    def forward(self, input_ids, attention_mask, visible_matrix, soft_positions):
        outputs = self.transformer(input_ids, attention_mask=attention_mask)
        # print("After transformer:", outputs.logits if hasattr(outputs, 'logits') else outputs.hidden_states[-1])

        hidden_states = outputs.hidden_states[-1]
        # print("hidden_states check NaN:", torch.isnan(hidden_states).any())
        soft_position_embeds = self.soft_position_embeddings(soft_positions)

        normalized_hidden_states = self.layer_norm(hidden_states)
        # print("normalized hidden_states check NaN:", torch.isnan(normalized_hidden_states).any())

        masked_outputs = self.masked_attention(normalized_hidden_states, attention_mask, visible_matrix)
        # print("masked_outputs check NaN:", torch.isnan(masked_outputs).any())

        pooled_output = masked_outputs.mean(dim=1)
        # print("pooled_output check NaN:", torch.isnan(pooled_output).any())

        logits = self.classifier(pooled_output)
        # print("classifier logits check NaN:", torch.isnan(logits).any())
        
        return logits

In [8]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    visible_matrices = [item['visible_matrix'] for item in batch]
    soft_positions = [item['soft_positions'] for item in batch]
    labels = [item['label'] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    visible_matrices = pad_sequence(visible_matrices, batch_first=True, padding_value=0)
    soft_positions = pad_sequence(soft_positions, batch_first=True, padding_value=0)
    labels = torch.stack(labels)  # stack them into a single tensor

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'visible_matrix': visible_matrices,
        'soft_positions': soft_positions,
        'labels': labels
    }

In [9]:
import os 

root_path = "cleaned_full_isear_data_with_sentiment.csv"
df = pd.read_csv(os.path.join(root_path))

In [10]:
unique_labels = sorted(df['label'].unique())
label_mapping = {original_label: idx for idx, original_label in enumerate(unique_labels)}
df['label'] = df['label'].map(label_mapping)

# Print to verify the mapping
print("Label mapping:", label_mapping)
print("Mapped labels:", df['label'].unique())

num_labels = len(unique_labels)
print(num_labels)

Label mapping: {2: 0, 11: 1, 14: 2, 17: 3, 25: 4, 28: 5, 29: 6}
Mapped labels: [3 2 0 4 1 5 6]
7


In [11]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# 1. Load the SenticNet Data
senticnet_path = "senticnet/senticnet.csv"
senticnet_data = load_senticnet(senticnet_path)

# 2. Load and Split the Dataset
data_path = "cleaned_full_isear_data_with_sentiment.csv"
full_data = pd.read_csv(data_path)

# Split into train, validation, and test sets
train_data, test_data = train_test_split(full_data, test_size=0.2, random_state=42)
train_data, dev_data = train_test_split(train_data, test_size=0.1, random_state=42)

print(f"Train size: {len(train_data)}, Validation size: {len(dev_data)}, Test size: {len(test_data)}")

#Tokenizer
pretrained_model_path = "results/checkpoint-3250"
tokenizer = RobertaTokenizer.from_pretrained(pretrained_model_path)
transformer = RobertaForSequenceClassification.from_pretrained(
    pretrained_model_path, num_labels=num_labels, output_hidden_states=True
)

#Create K_BERTDataset Instances
train_dataset = K_BERTDataset(train_data, tokenizer, senticnet_data)
dev_dataset = K_BERTDataset(dev_data, tokenizer, senticnet_data)
test_dataset = K_BERTDataset(test_data, tokenizer, senticnet_data)

#Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

num_attention_heads = 12  # Default number of attention heads for Roberta


# Initialize the model
model = K_RoBERTa(
    transformer = RobertaForSequenceClassification.from_pretrained(
    pretrained_model_path, num_labels=num_labels, output_hidden_states=True
),
    num_attention_heads=num_attention_heads,
    num_labels=num_labels,
    hidden_size = transformer.config.hidden_size
)

#Optimizer and Loss Function
optimizer = AdamW(
    model.parameters(),
    lr=3e-7,  # Reduced learning rate for stability
    weight_decay=1e-4,
    eps=1e-8
)

criterion = nn.CrossEntropyLoss()

#Training Mode
model.train()


Train size: 5334, Validation size: 593, Test size: 1482


/home/hice1/syoon333/.local/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


K_RoBERTa(
  (transformer): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense)

In [12]:
model = model.to(device)

In [13]:
#torch.autograd.set_detect_anomaly(True)

for epoch in range(50):
    total_loss = 0
    model.train()
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        visible_matrix = batch['visible_matrix'].to(device)
        soft_positions = batch['soft_positions'].to(device)
        labels = batch['labels'].to(device)

#         if torch.isnan(input_ids).any() or torch.isnan(visible_matrix).any():
#             print("NaN in inputs before forward pass")
        
        optimizer.zero_grad()
        
        logits = model(input_ids, attention_mask, visible_matrix, soft_positions)
        
#         if torch.isnan(logits).any():
#             print("NaN detected in logits before loss computation")
        
        loss = criterion(logits, labels)

        # Gradient clipping to avoid parameter explosion
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1} Average Loss: {avg_loss}")


Training Epoch 1: 100%|██████████| 334/334 [00:12<00:00, 26.05it/s]


Epoch 1 Average Loss: 1.742286432645992


Training Epoch 2: 100%|██████████| 334/334 [00:12<00:00, 27.34it/s]


Epoch 2 Average Loss: 1.3846035565801724


Training Epoch 3: 100%|██████████| 334/334 [00:12<00:00, 27.32it/s]


...

Epoch 48 Average Loss: 0.25702353410177425


Training Epoch 49: 100%|██████████| 334/334 [00:12<00:00, 27.14it/s]


Epoch 49 Average Loss: 0.25774772021874875


Training Epoch 50: 100%|██████████| 334/334 [00:12<00:00, 27.15it/s]

Epoch 50 Average Loss: 0.2617207909571732


In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", unit="batch"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            visible_matrix = batch['visible_matrix'].to(device)
            soft_positions = batch['soft_positions'].to(device)
            labels = batch['labels'].to(device)
            
            
            logits = model(input_ids, attention_mask, visible_matrix, soft_positions)

            preds = torch.argmax(logits, dim=1)

            all_labels.extend(labels.tolist())
            all_preds.extend(preds.tolist())
      

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted') 
    recall = recall_score(all_labels, all_preds, average='weighted')        
    f1 = f1_score(all_labels, all_preds, average='weighted')

    return accuracy, precision, recall, f1

In [15]:
kbert_accuracy, kbert_precision, kbert_recall, kbert_f1 = evaluate_model(model, test_loader)
print(f"K-BERT with SenticNet Accuracy: {kbert_accuracy * 100:.2f}%")
print(f"K-BERT Mean Precision: {kbert_precision * 100:.2f}%")
print(f"K-BERT Mean Recall: {kbert_recall * 100:.2f}%")
print(f"K-BERT Mean F1 Score: {kbert_f1 * 100:.2f}%")

Evaluating: 100%|██████████| 93/93 [00:01<00:00, 59.21batch/s]

K-BERT with SenticNet Accuracy: 88.06%
K-BERT Mean Precision: 88.09%
K-BERT Mean Recall: 88.06%
K-BERT Mean F1 Score: 88.06%
